In [ ]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
(path/"images").ls()

In [ ]:
fname = (path/"images").ls()[0]; fname

In [ ]:
re.findall(r'(.+)_\d+.jpg$', fname.name)

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items=get_image_files,
                splitter=RandomSplitter(seed=42),
                get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                item_tfms=Resize(460),
                batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images")

In [ ]:
dls.show_batch(nrows=1, ncols=5)

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

In [ ]:
x,y = dls.one_batch()

In [ ]:
y

In [ ]:
x[0],y[0]

In [ ]:
preds,_=learn.get_preds(dl=[(x,y)])
preds[0]

# Log Likelihood

In [ ]:
torch.random.manual_seed(42);
acts = torch.randn((6,2))*2
acts

In [ ]:
acts.sigmoid()

In [ ]:
sm_acts = torch.softmax(acts, dim=1)
sm_acts

In [ ]:
targ = tensor([0,1,0,1,1,0])

In [ ]:
idx = range(6)
sm_acts[idx, targ]

In [ ]:
from IPython.display import HTML
df = pd.DataFrame(sm_acts, columns=["3","7"])
df['targ'] = targ
df['idx'] = idx
df['result'] = sm_acts[range(6), targ]
df['loss'] = -torch.log(tensor(df['result']))
t = df.style.hide()
#To have html code compatible with our script
html = t._repr_html_()#.split('')[1]
html = re.sub(r'', r'', html)
display(HTML(html))

Legative Loss Likelihood

In [ ]:
-sm_acts[idx, targ]


Using pytorch:

In [ ]:
F.nll_loss(sm_acts, targ, reduction='none')


It doesn't take a log, despite the name. It's designed to be used after `log_softmax`

Cross entropy loss is equal to our negative log loss column (which, in practice, actually does `log_softmax` and then `nll_loss`). `reduction=none` does not take the mean value of all the elements.

In [ ]:
nn.CrossEntropyLoss(reduction='none')(acts, targ)

Let's try taking the mean:

In [ ]:
nn.CrossEntropyLoss()(acts, targ)

And compute the mean manually, which will give the same number

In [ ]:
tensor([0.5067, 0.6973, 2.0160, 5.6958, 0.9062, 1.0048]).mean()

# Model Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=5)

# Improving the model

## Learning rate


In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1, base_lr=0.1)

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
lr_min, lr_steep

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2, base_lr=0.00363)

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(6, lr_max=1e-5)

## Discriminative Learning Rates

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))

In [ ]:
learn.recorder.plot_loss()

## Selecting a larger net, resnet50 and using mixed-precision training

Using half-precision to speed up training

In [ ]:
from fastai.callback.fp16 import *

Using resnet50 instead of resnet34 that we used before, and adding `to_fp16`

In [ ]:
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fine_tune(6, freeze_epochs=3)

The minimum error rate is 0.056 using resnet50, while it's 0.058 with resnet34 - slightly better. I don't think using larget net is worth it. I would re-train the smaller one for 10 epochs instead of 12

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)
learn.unfreeze()
learn.fit_one_cycle(10, lr_max=slice(1e-6,1e-4))

that's worse than it was before :(. Let's try to use resnet50 with mixed-precision training and determine the best learning rate

In [ ]:
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fit_one_cycle(3, 5e-3)
learn.unfreeze()
learn.lr_find()

In [ ]:
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fit_one_cycle(3, 5e-3)
learn.unfreeze()
learn.fit_one_cycle(10, lr_max=slice(1e-6,1e-5))

This is worse than resnet34 or resnet50 trained with `fine_tune`. Let's try resnet34 and 50 to train with `fine_tune` again.

In [ ]:
learn50 = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn50.fine_tune(6, freeze_epochs=3)

In [ ]:
learn34 = vision_learner(dls, resnet34, metrics=error_rate)
learn34.fine_tune(6, freeze_epochs=3)

Achieved an identical `error_rate` or 0.0629 for both resnet34 ane 50! Again, this is much worse than 0.056834 when I train resnet50, above. Why is that?

In [ ]:
learn50_2 = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn50_2.fine_tune(6, freeze_epochs=3)

Not even close to 0.056 achieved on the first run. Worse that 0.062 that we achieved above. Every subsequent run is worse than the one before.

In [ ]:
learn50_3 = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn50_3.fine_tune(6, freeze_epochs=3)

0.058 is better than 0.065 but not quite as good as 0.056 achived on the first run of the resnet50. Let's plot learning rates for the last 2 runs

In [ ]:
learn50_2.recorder.plot_loss()

In [ ]:
learn50_3.recorder.plot_loss()